In [1]:
import pandas as pd
# import matplotlib as plt
# import numpy as np

In [2]:
fileName = "Vehicle Check/Northern Branch Phase II Debris Removal Ops.xlsx"
df = pd.read_excel(fileName)

# Variables to use

In [3]:
status = ("Withdrawal", "Ineligible") # TODO: make apart of our search 
active_crews = ["CREW#101", "CREW#102", "CREW#203", "CREW#301", "CREW#404", "CREW#501", "CREW#406", "CREW#304", "CREW#701",
                "CREW#702", "CREW#703", "CREW#704", "CREW#705", "CREW#706",
                "CREW#707" "CREW#708", "CREW#801", "CREW#802", "CREW#803", "CREW#805"]

In [4]:
df.columns.tolist()


['APN',
 'Structural Status',
 'Haz Tree Only',
 '# of Trees',
 'Haz Trees Assessment',
 'Haz Trees',
 'Hazard Tree Status',
 'Street #',
 'Unit #',
 'Street Name',
 'City',
 'County',
 'Division',
 'Hold Reason',
 'Hold',
 'Gate Locked?',
 'Code/Key Location',
 'ROE Verified',
 'Declaration',
 'Property Type',
 'Commercial Notes',
 'Scheduled',
 'Consultant',
 'Debris Crew',
 'Debris Crew Leader/Crew#',
 'Debris Crew WO#',
 'ROE Date',
 'Priority',
 'Withdrawal',
 'Not Eligible',
 'Reason for Ineligibility',
 'Determinant for Ineligibility',
 'Safety Concern Property',
 'Phase 1 Complete',
 'Needs Milk Run or Phase 1 Revisit',
 'Milk Run/Phase 1 Revisit Complete',
 'Access Issues',
 'EHP Attention Required',
 'EHP Attn Notes',
 'Monitoring Needed',
 'Monitoring Needed Description',
 'Bio Assessment Notes',
 'Arch Assessment Notes',
 'CalOES Review Notes',
 'Permit Needed',
 'Permit Description',
 'Owner Name',
 'Contact Phone',
 'Contact Email',
 'Site Assessment',
 'RAD/MERC Sweep',


# Get average from start and finish dates of crews
# Steps:
## -Finish days minus start day
## -Change data types from timedate to float
## -Make data frame only show the columns we need
## -Group by county and get the sum, count, and mean of times debris was done on propeties

In [5]:
df[['Debris Finish', 'Debris Start']].dtypes

Debris Finish    datetime64[ns]
Debris Start     datetime64[ns]
dtype: object

# Pick day you need below
# And filter out "Ineligible", "Withdrawal" properties

In [8]:
df.loc[:, ['Debris Start', 'Debris Finish']] = df[(df.loc[:, 'Debris Start']> "2021-05-01") & (df.loc[:, 'Debris Finish'] <= "2021-06-19")]

df.loc[:, "Structural Status"] = df[~df.loc[:, "Structural Status"].isin(status)]

In [ ]:
df.loc[:, "duration"] = df.loc[:, 'Debris Finish'] - df.loc[:, 'Debris Start']

df["duration"] = df["duration"].dt.days # convert 'duration' column to a float instead of timedelta64[ns] 

# removal all other columsn from data frame except the following:
df = df[['APN', "Structural Status", 'County', 'Debris Crew', 'Debris Crew WO#', 'Debris Start', 'Debris Finish', 'duration']] 
df['Debris Crew WO#'] = df['Debris Crew WO#'].str.extract('(CREW ?# ?\d+)') ### LEARN THIS YESTERDAY!!!!!!


df.groupby(['County', 'Debris Crew WO#'] ).duration.agg(['sum', 'count', 'mean'])
# dffilt.to_excel("Date Averages formatting issue.xlsx") # put it in an excel doc

# Take out spaces then
# Filter for crews needed

In [ ]:
df.loc[:, "Debris Crew WO#"] = df.loc[:, "Debris Crew WO#"].str.replace(" ", "")

In [ ]:
df.loc[:, 'Debris Crew WO#'] = df[df.loc[:, 'Debris Crew WO#'].isin(active_crews)]

In [ ]:
dffilt = df.groupby(['County', 'Debris Crew WO#'])[['duration']].mean()
dffilt.to_excel("Overview test.xlsx")
dffilt

# See all columns on screen. *Takes more memory (RAM)

In [ ]:
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 2500)

# Need to filter on the  'Debris Crew WO#' for all chars after crew is read. 
## Eaiser way is to grab last 11 chars but not accurate

In [ ]:
df['Debris Crew WO#'] = df['Debris Crew WO#'].str.extract('(CREW ?# ?\d+)')
df.groupby(['APN', 'Debris Crew WO#'])["duration"].mean()

# See if we can replace all values in Debris Crew WO# with just the actie crews

In [ ]:
df.loc[:, ['Place Holder']] = df.loc[:, 'Debris Crew WO#'].str[28:] # not very effective due to formatting
df.loc[:, ['Place Holder']]

In [ ]:
df.loc[:, ['Place Holder']] = df.loc[:, 'Debris Crew WO#'].str[26:]
df.loc[:, ['Place Holder']] = df.loc[:, ['Place Holder']].split().join(" ")


# Method #1 referring to a list: How to grab just CREW # and number from text
## You can use str.extract() with the capture group being a joined list with join('|'). The | symbol is for OR and allows you to search multiple values simultaneously for each row. Capture groups require parentheses around them which is why I add parentheses as strings before and after.

In [ ]:
df['Debris Crew WO#'] = df['Debris Crew WO#'].str.extract(('(' + '|'.join(active_crews) + ')'))
df['Debris Crew WO#']

# Method #2 
## Extracting based off a regex pattern and ignoring the list. A ? after a space means that the space is optional. Instead of a space you can also do \s or \s+ for multiple spaces. \d+ means consecutive numbers. If there are commas in the numbers, the regex is slightly different:

In [ ]:
df['Debris Crew WO#'] = df['Debris Crew WO#'].str.extract('(CREW ?# ?\d+)')
df['Debris Crew WO#']

In [1]:
df['test 2'] = df['Debris Crew WO#'].str.extract('(CREW ?# ?\d)')
df['test2']

NameError: name 'df' is not defined